In [ ]:
# Visualize the results from a pde1bvp_coupled_pinn.py run.

In [ ]:
from importlib import import_module
import math as m
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [ ]:
# Specify the run ID (aka problem name).
runid = "eplasma1"

# Add the subdirectory for the run results to the module search path.
run_path = os.path.join(".", runid)
sys.path.append(run_path)

# Import the problem definition from the run results directory.
p = import_module(runid)

# Read the run hyperparameters from the run results directory.
import hyperparameters as hp

In [ ]:
# Load the training points.
xy_train = np.loadtxt(os.path.join(runid, "xy_train.dat"))

In [ ]:
# Load the loss function histories.
losses_model_all = np.loadtxt(os.path.join(runid, "losses_model_all.dat"))
losses_model_bc = np.loadtxt(os.path.join(runid, "losses_model_bc.dat"))
losses_model = np.loadtxt(os.path.join(runid, "losses_model.dat"))
losses_all = np.loadtxt(os.path.join(runid, "losses_all.dat"))
losses_bc = np.loadtxt(os.path.join(runid, "losses_bc.dat"))
losses = np.loadtxt(os.path.join(runid, "losses.dat"))


In [ ]:
# Load the model values.
Y = []
delY = []
for i in range(len(p.variable_names)):
    var_name = p.variable_names[i]
    Y.append(np.loadtxt(os.path.join(runid, "%s_train.dat" % var_name)))
    delY.append(np.loadtxt(os.path.join(runid, "del_%s_train.dat" % var_name)))


In [ ]:
# Plot the loss function histories for the individual models.
n_models = len(p.variable_names)

# Compute the number of rows for the 2-per-row plot.
n_rows = m.ceil(n_models/2)

# Plot the loss history for each model.
plt.figure(figsize=(12, 6))
for i in range(n_models):
    plt.subplot(n_rows, 2, i + 1)
    variable_name = p.variable_names[i]
    plt.semilogy(losses_model_all[:, i], label="$L_{all,%s}$" % variable_name)
    plt.semilogy(losses_model_bc[:, i], label="$L_{bc,%s}$" % variable_name)
    plt.semilogy(losses_model[:, i], label="$L_{%s}$" % variable_name)
    plt.title(variable_name)
    plt.legend()
plt.suptitle("Loss function histories by model")
plt.show()

In [ ]:
# Plot the total loss function history.
plt.semilogy(losses_all, label="$L_{all}$")
plt.semilogy(losses_bc, label="$L_{bc}$")
plt.semilogy(losses, label="$L$")
plt.xlabel("Epoch")
plt.ylabel("Loss function")
plt.legend()
plt.grid()
plt.title("Loss function evolution for %s" % runid)
plt.show()

In [ ]:
# Load the training points.
xt_train = np.loadtxt(os.path.join(runid, "xy_train.dat"))
x_train = xt_train[:, 0]
t_train = xt_train[:, 1]

# Create plot labels from the training points.
x_labels = ["%.1f" % x for x in x_train[0:-1:hp.ny_train]]
t_labels = ["%.1f" % t for t in t_train[0:hp.nx_train]]
t_labels.reverse()

# Plot the model values.
plt.figure(figsize=(12, 6))
for i in range(n_models):
    plt.subplot(n_rows, 2, i + 1)
    variable_name = p.variable_names[i]
    # For a Seaborn heat map, reshape as (n_x, n_y), then transpose, then flip.
    Z = np.flip(Y[i].reshape(hp.nx_train, hp.ny_train).T, axis=0)
    ax = sns.heatmap(Z, vmin=-0.1, vmax=0.1)
    ax.set_xlabel("x")
    ax.set_ylabel("t")
    plt.title(variable_name)
plt.suptitle("Model values")
plt.show()

In [ ]:
# Compute wavenumber and angular frequency.
kx =p.kx
w = p.electron_plasma_wave_angular_frequency(p.n0, p.T, kx)

In [ ]:
# Compute analytical solutions for n1, u1x, E1x.
n1_a = p.n10*np.sin(kx*x_train - w*t_train)
u1x_a = p.u1x0*np.sin(kx*x_train - w*t_train)

In [ ]:
# Plot the analytical solutions.
plt.figure(figsize=(12, 6))

# n1_a
plt.subplot(1, 2, 1)
variable_name = p.variable_names[0]
# For a Seaborn heat map, reshape as (n_x, n_y), then transpose, then flip.
Z = np.flip(n1_a.reshape(hp.nx_train, hp.ny_train).T, axis=0)
ax = sns.heatmap(Z, vmin=-0.1, vmax=0.1)
ax.set_xlabel("x")
ax.set_ylabel("t")
plt.title(variable_name)

plt.subplot(1, 2, 2)
# u1x_a
Z = np.flip(u1x_a.reshape(hp.nx_train, hp.ny_train).T, axis=0)
ax = sns.heatmap(Z, vmin=-0.1, vmax=0.1)
ax.set_xlabel("x")
ax.set_ylabel("t")
variable_name = p.variable_names[1]
plt.title(variable_name)

plt.suptitle("Analytical values")
plt.show()

In [ ]:
# Compute the error in the predicted solutions relative to the analytical solutions.
n1_err = Y[0] - n1_a
u1x_err = Y[1] - u1x_a

In [ ]:
# Plot the error in the predicted solutions.
plt.figure(figsize=(12, 6))

# v1y_err
plt.subplot(1, 2, 1)
variable_name = p.variable_names[0]
# For a Seaborn heat map, reshape as (n_x, n_y), then transpose, then flip.
Z = np.flip(n1_err.reshape(hp.nx_train, hp.ny_train).T, axis=0)
ax = sns.heatmap(Z, vmin=-0.1, vmax=0.1)
ax.set_xlabel("x")
ax.set_ylabel("t")
plt.title(variable_name)

plt.subplot(1, 2, 2)
# B1y_err
Z = np.flip(u1x_err.reshape(hp.nx_train, hp.ny_train).T, axis=0)
ax = sns.heatmap(Z, vmin=-0.1, vmax=0.1)
ax.set_xlabel("x")
ax.set_ylabel("t")
variable_name = p.variable_names[1]
plt.title(variable_name)

plt.suptitle("Prediction errors")
plt.show()

In [ ]:
# Plot predicted, analytical, and error in n1 at t = t0 and t = t1.
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
it = 0
x = x_train[it::hp.ny_train]
y = Y[0][it::hp.ny_train]
y_a = n1_a[it::hp.ny_train]
y_err = n1_err[it::hp.ny_train]
rms_err = np.sqrt(np.sum(y_err**2)/len(y_err))
plt.plot(x, y, label="$n_{1,p}$")
plt.plot(x, y_a, label="$n_{1,a}$")
plt.plot(x, y_err, label="$n_{1,err}$")
plt.grid()
plt.legend()
plt.title("Error in predicted $n_{1}$ at t = %f, RMS = %f" %
          (p.t0 + it/(hp.ny_train - 1)*(p.t1 - p.t0), rms_err))
plt.xlabel("x")
plt.ylabel("$n_{1}$")

plt.subplot(1, 2, 2)
it = hp.ny_train - 1
x = x_train[it::hp.ny_train]
y = Y[0][it::hp.ny_train]
y_a = n1_a[it::hp.ny_train]
y_err = n1_err[it::hp.ny_train]
rms_err = np.sqrt(np.sum(y_err**2)/len(y_err))
plt.plot(x, y, label="$n_{1,p}$")
plt.plot(x, y_a, label="$n_{1,a}$")
plt.plot(x, y_err, label="$n_{1,err}$")
plt.grid()
plt.legend()
plt.title("Error in predicted $n_{1}$ at t = %f, RMS = %f" %
          (p.t0 + it/(hp.ny_train - 1)*(p.t1 - p.t0), rms_err))
plt.xlabel("x")
plt.ylabel("$n_{1}$")

plt.show()

In [ ]:
# Plot predicted, analytical, and error in u1x at t = t0 and t = t1.
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
it = 0
x = x_train[it::hp.ny_train]
y = Y[1][it::hp.ny_train]
y_a = u1x_a[it::hp.ny_train]
y_err = u1x_err[it::hp.ny_train]
rms_err = np.sqrt(np.sum(y_err**2)/len(y_err))
plt.plot(x, y, label="$u_{1x,p}$")
plt.plot(x, y_a, label="$u_{1x,a}$")
plt.plot(x, y_err, label="$u_{1x,err}$")
plt.grid()
plt.legend()
plt.title("Error in predicted $u_{1x}$ at t = %f, RMS = %f" %
          (p.t0 + it/(hp.ny_train - 1)*(p.t1 - p.t0), rms_err))
plt.xlabel("x")
plt.ylabel("$u_{1x}$")

plt.subplot(1, 2, 2)
it = hp.ny_train - 1
x = x_train[it::hp.ny_train]
y = Y[1][it::hp.ny_train]
y_a = u1x_a[it::hp.ny_train]
y_err = u1x_err[it::hp.ny_train]
rms_err = np.sqrt(np.sum(y_err**2)/len(y_err))
plt.plot(x, y, label="$u_{1x,p}$")
plt.plot(x, y_a, label="$u_{1x,a}$")
plt.plot(x, y_err, label="$u_{1x,err}$")
plt.grid()
plt.legend()
plt.title("Error in predicted $u_{1x}$ at t = %f, RMS = %f" %
          (p.t0 + it/(hp.ny_train - 1)*(p.t1 - p.t0), rms_err))
plt.xlabel("x")
plt.ylabel("$u_{1x}$")

plt.show()